In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:
df = pd.read_csv("국민건강보험공단_건강검진정보_20221231.CSV", encoding='cp949')

In [6]:
df.columns

Index(['기준년도', '가입자일련번호', '시도코드', '성별', '연령대코드(5세단위)', '신장(5cm단위)',
       '체중(5kg단위)', '허리둘레', '시력(좌)', '시력(우)', '청력(좌)', '청력(우)', '수축기혈압',
       '이완기혈압', '식전혈당(공복혈당)', '총콜레스테롤', '트리글리세라이드', 'HDL콜레스테롤', 'LDL콜레스테롤',
       '혈색소', '요단백', '혈청크레아티닌', '혈청지오티(AST)', '혈청지피티(ALT)', '감마지티피', '흡연상태',
       '음주여부', '구강검진수검여부', '치아우식증유무', '치석'],
      dtype='object')

In [8]:
sample_df=df[['신장(5cm단위)', '성별', '체중(5kg단위)', '음주여부']]

In [9]:
sample_df[:10]

,신장(5cm단위),성별,체중(5kg단위),음주여부
0,160,2,75,0.0
1,150,2,45,1.0
2,150,2,60,1.0
3,160,2,50,1.0
4,160,2,55,1.0
5,170,1,85,1.0
6,160,2,40,1.0
7,180,1,65,1.0
8,155,2,50,1.0
9,165,2,55,1.0


In [10]:
# info 정보로 결측치 확인
print("Info 정보 확인")
sample_df.info()

Info 정보 확인
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   신장(5cm단위)  1000000 non-null  int64  
 1   성별         1000000 non-null  int64  
 2   체중(5kg단위)  1000000 non-null  int64  
 3   음주여부       999927 non-null   float64
dtypes: float64(1), int64(3)
memory usage: 30.5 MB


In [12]:
sample = sample_df.dropna()

In [13]:
print("Drop 후 Info 정보 확인")
sample.info()

Drop 후 Info 정보 확인
<class 'pandas.core.frame.DataFrame'>
Index: 999927 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   신장(5cm단위)  999927 non-null  int64  
 1   성별         999927 non-null  int64  
 2   체중(5kg단위)  999927 non-null  int64  
 3   음주여부       999927 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 38.1 MB


In [15]:
# 원핫 인코딩을 위해 데이터 object 형태로 변경하기
sample = sample.astype('str')

# label(결과,Y) 생성하기
y = sample.음주여부

# 음주여부 학습 데이터 구성하기
X = sample.drop('음주여부', axis=1)

# label 데이터의 편향성 확인하기
y.value_counts()

음주여부
1.0    655146
0.0    344781
Name: count, dtype: int64

In [16]:
# 학습/검증 데이터 분리하기
x_train, x_valid, y_train, y_valid = train_test_split(
    X, y,
    test_size=0.2,
    shuffle=True,
    random_state=34
)

In [17]:
# 의사결정나무 모델 불러오기
from sklearn.tree import DecisionTreeClassifier

In [18]:
# 의사결정나무 모델 생성하기
dt = DecisionTreeClassifier(random_state=1001, max_depth=2)

In [19]:
# 의사결정나무 학습하기
dt_model = dt.fit(x_train, y_train)

# 학습 데이터 정확도 확인하기
print("학습 정확도=", dt_model.score(x_train, y_train))

# 검증 데이터 정확도 확인하기
print("검증 정확도=", dt_model.score(x_valid, y_valid))

학습 정확도= 0.6985977715856544
검증 정확도= 0.7003840268818817


In [4]:
!pip install graphviz

In [20]:
import matplotlib.pyplot as plt
import graphviz
from sklearn.tree import export_graphviz

In [23]:
# 의사결정나무 그래프 그리기
# 그래프 생성하기
# 에러발생 사이트 참조: https://highschoolfree.tistory.com/76
tree_graph = graphviz.Source(export_graphviz(dt_model,
                                            class_names=["X","O"],
                                            feature_names=['height', 'sex', 'weight'],
                                            impurity=True, # 불순도 표시 여부
                                            filled=True)) # 각 노드의 색상을 클래스에 따라 다르게 채울지 여부
# 그래프 출력
tree_graph

# 그래프의 각 노드는 노드 분할 조건, 불순도, 각 클래스에 속한 샘플 수, 노드가 속하는 클래스 포함

# 그래프 파일 저장하기
tree_graph.render('tree_depth5', format="png")

'tree_depth5.png'

In [ ]:
# 노드 맨 첫 줄에 나오는 분할 조건은 자식 노드를 만들기 위한 규칙 조건
# 두 번째 줄의 지니계수는 불순도 지표
# 의사결정나무는 이 지니계수를 낮추는 방향으로 가지를 분할

In [25]:
# 불순도 알아보기
# 불순도 함수 생성하기
def gini(x):
    n=x.sum()
    gini_sum=0
    for key in x.keys():
        gini_sum=gini_sum+(x[key]/n) * (x[key]/n)
    gini=1-gini_sum
    return gini

# 데이터 준비하기(불순도 예시)
과일바구니1 = ['사과']*9
과일바구니2 = ['사과', '바나나', '사과', '바나나', '바나나', '바나나', '복숭아', '복숭아', '복숭아']
과일바구니3 = ['사과', '바나나', '사과', '바나나', '사과', '복숭아', '복숭아', '사과', '복숭아']
print(round(gini(pd.DataFrame(과일바구니1).value_counts()),3))
print(round(gini(pd.DataFrame(과일바구니2).value_counts()),3))
print(round(gini(pd.DataFrame(과일바구니3).value_counts()),3))

0.0
0.642
0.642


In [ ]:
# 의사결정나무는 지니계수와 같은 불순도 지표를 기반으로 각 노드에서 불순도를 최소화하는 최적의 분할 조건을 구하며 데이터를 학습